In [45]:
# mengambil dataset
df = pd.read_csv('https://drive.google.com/uc?id=1XZbZ5Z7MBXYaR3UO1HbQ4ncPi4R3HsyF')
df.head()

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9


In [ ]:
# mengecek column dari dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  int64  
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 468.9 KB


In [ ]:
# karena ada column yang bertipe object, maka diubah menjadi numerik
# cara pertama (manual)
df.name[df.name == 'grapefruit'] = 0
df.name[df.name == 'orange'] = 1

<ipython-input-10-2ccc7221f1a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'grapefruit'] = 0
<ipython-input-10-2ccc7221f1a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'orange'] = 1


In [46]:
# karena ada column yang bertipe object, maka diubah menjadi numerik
# cara kedua (otomatis)
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['name'] = label_encoder.fit_transform(df['name'])
print(df)

      name  diameter  weight  red  green  blue
0        1      2.96   86.76  172     85     2
1        1      3.91   88.05  166     78     3
2        1      4.42   95.17  156     81     2
3        1      4.47   95.60  163     81     4
4        1      4.48   95.76  161     72     9
...    ...       ...     ...  ...    ...   ...
9995     0     15.35  253.89  149     77    20
9996     0     15.41  254.67  148     68     7
9997     0     15.59  256.50  168     82    20
9998     0     15.92  260.14  142     72    11
9999     0     16.45  261.51  152     74     2

[10000 rows x 6 columns]


In [47]:
# dataset harus diconvert ke array agar bisa diproses oleh model
dataset = df.values
print(dataset)

[[  1.     2.96  86.76 172.    85.     2.  ]
 [  1.     3.91  88.05 166.    78.     3.  ]
 [  1.     4.42  95.17 156.    81.     2.  ]
 ...
 [  0.    15.59 256.5  168.    82.    20.  ]
 [  0.    15.92 260.14 142.    72.    11.  ]
 [  0.    16.45 261.51 152.    74.     2.  ]]


In [48]:
# Kemudian kita perlu memisahkan antara atribut independen dan atribut dependen (label) pada dataset.
# Dari eksplorasi data kita sebelumnya, dapat diketahui bahwa 5 kolom terakhir adalah kolom atribut independen.

# pilih 5 kolom terakhir sebagai atribut
# bilangan sebelum koma untuk memilih baris pada dataframe
# bilangan setelah koma untuk memilih kolom pada dataframe

X = dataset[:,1:6]
print(X)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

# Jangan lupa untuk memisahkan label dari dataset. Pada dataset kita, label terdapat pada kolom pertama.
# Jalankan kode di bawah untuk membuat sebuah array numpy yang hanya berisi label kita.

y = dataset[:,0]
print(y)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

[[  2.96  86.76 172.    85.     2.  ]
 [  3.91  88.05 166.    78.     3.  ]
 [  4.42  95.17 156.    81.     2.  ]
 ...
 [ 15.59 256.5  168.    82.    20.  ]
 [ 15.92 260.14 142.    72.    11.  ]
 [ 16.45 261.51 152.    74.     2.  ]]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[1. 1. 1. ... 0. 0. 0.]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [49]:
# Normalisasi data
# valuenya akan menjadi 0-1
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

In [50]:
# pisahkan data training dan testing
# data test = 30%, data training = 70%
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

In [51]:
# mulai membuat neural network
from keras.models import Sequential
from keras.layers import Dense

In [53]:
# Model Sequential dengan 3 layer
# 2 layer pertama adalah activation function relu
# layer ke-3 untuk menentukan probabilitas
# layer ke-3 dipilih sigmoid karena menentukan probabilitas 0 sampai 1, cocok untuk 2 kelas/data (biner)
model = Sequential([
                    Dense(32, activation='relu', input_shape=(5,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')])

In [54]:
# menentukan optimizer dan loss function
# dipilih binary_crossentropy karena cocok untuk 2 kelas/data (biner)
# metrics accuracy dipilih untuk menampilkan akurasi pada setiap proses pelatihan model
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [55]:
# memulai tahap pelatihan model
model.fit(X_train, Y_train, epochs=100)

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5851 - loss: 0.6894
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8919 - loss: 0.6469
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8991 - loss: 0.5886
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9223 - loss: 0.4999
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9314 - loss: 0.3974
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9270 - loss: 0.3163
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9290 - loss: 0.2594
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9281 - loss: 0.2298
Epoch 9/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9178 - loss: 0.2261
Epoch 10/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9244 - loss: 0.2039
Epoch 11/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9252 - loss: 0.1944
Epoch 12/100
219/219 ━━━━━━━━━━━━━━━━━━━━

In [56]:
# mengevaluasi performa accuracy model
# apakah overfit, underfit, atau goodfit.
model.evaluate(X_test, Y_test, batch_size=1)

3000/3000 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9238 - loss: 0.1752


[0.1720467358827591, 0.9269999861717224]